In [4]:
import requests

auth_url = "https://signin.tradestation.com/authorize"

public_key = "r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4"
private_key = "hFBk8xgV_UGJEUFnxVW-AFz6YToqZwdvM-48x5wLUQhzKiR99r2w780hL0giBfvd"

params = {"response_type": "code", "client_id": public_key, "redirect_uri": "http://localhost", "audience": "https://api.tradestation.com", "state": "STATE", 
        "scope": "openid offline_access profile MarketData ReadAccount Trade Crypto Matrix OptionSpreads"}

response = requests.get(auth_url, params=params)
response.history
for resp in response.history:
        print(resp.status_code, resp.url)

302 https://signin.tradestation.com/authorize?response_type=code&client_id=r4bJ08Nbz9f8b6djDhoyCmazNnrrLFL4&redirect_uri=http%3A%2F%2Flocalhost&audience=https%3A%2F%2Fapi.tradestation.com&state=STATE&scope=openid+offline_access+profile+MarketData+ReadAccount+Trade+Crypto+Matrix+OptionSpreads


In [ ]:
import requests

token_url = "https://signin.tradestation.com/oauth/token"

authorization_code = "xxMZvFaqtX95pCIj"

headers = {"content-type": "application/x-www-form-urlencoded"}
data = {"grant_type": "authorization_code", "client_id": public_key, "client_secret": private_key, "code": authorization_code, "redirect_uri": "http://localhost"}

r = requests.post(token_url, headers=headers, data=data).json()
print(r)

access_token = r["access_token"]
refresh_token = r["refresh_token"]
id_token = r["id_token"]

In [2]:
import requests
refresh_token = "lzijZJe0VUnjFEzq6dqWK-Q3rqhzyoq7kkrDkqVoJvoYn"
def refresh():
    token_url = "https://signin.tradestation.com/oauth/token"

    headers = {"content-type": "application/x-www-form-urlencoded"}
    data = {"grant_type": "refresh_token", "client_id": public_key, "client_secret": private_key, "refresh_token": refresh_token}

    r = requests.post(token_url, headers=headers, data=data).json()

    access_token = r["access_token"]
    return access_token

In [ ]:
import requests
import pandas as pd
import json
import csv

url = "https://api.tradestation.com/v3/marketdata/barcharts/TSLA"

headers = {"Authorization": f"Bearer {access_token}"}

params = {"unit": "Minute", "firstdate": "2016-12-05T21:00:00Z", "lastdate": "2016-12-06T21:00:00Z"}

response = requests.request("GET", url, headers=headers, stream=True, params=params)
print(response)
if response.status_code == 401:
    access_token = refresh()
    headers = {"Authorization": f"Bearer {access_token}"}
    response = requests.request("GET", url, headers=headers, stream=True, params=params)
    print(response)
response = response.json()
    
fieldnames = list(response["Bars"][0].keys())
with open("tradestation_data/tsla_2016_12_05_2016_12_06.csv", "w", newline="") as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    writer.writeheader()
    for row in response["Bars"]:
        writer.writerow(row)

In [ ]:
def stream_lvl1_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/quotes/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue   

            for line in response.iter_lines():
                if line:
                    row = json.loads(line)
                    if "Error" in row:
                        print(f"{symbol} {response} {response.text}")
                        continue
                    if "Heartbeat" in row:
                        continue
                    print(row)
        except Exception as e:
            print(f"{symbol} {e}")
            continue

In [ ]:
import os
import csv
import json
import pandas as pd
import datetime
import time

def stream_lvl2_quotes(symbol):
    url = f"https://api.tradestation.com/v3/marketdata/stream/marketdepth/aggregates/{symbol}"
    while True:
        try:
            access_token = refresh()
            headers = {"Authorization": f"Bearer {access_token}"}

            response = requests.get(url, headers=headers, stream=True)
            if response.status_code != 200:
                print(f"{symbol} {response} {response.text}")
                continue            
            
            for line in response.iter_lines():
                if line:
                    bids_and_asks = json.loads(line)
                    if "Heartbeat" in bids_and_asks:
                        continue
                    print(bids_and_asks)
        except Exception as e:
            print(f"{symbol} {e}")
            break
                    
symbols = "TSLA"
stream_lvl2_quotes(symbols)

In [16]:
from test_mongo import MongoExplorer
mongo = MongoExplorer()
mongo

ModuleNotFoundError: No module named 'pymongo'

In [14]:
import numpy as np
import pandas as pd
def calc_gain(symbol, date, price_dif_threshold, size_threshold):
    collection_1 = f"{symbol}_10sec_ts_lvl1"
    collection_2 = f"{symbol}_10sec_ts_lvl2"
    start = f'{date}T13:30:00Z'
    end = f'{date}T20:00:00Z'
    query = {'$and': [{'CurTime': {'$gte': start}}, {'CurTime': {'$lt': end}}]}
    df_1 = pd.DataFrame(list(mongo.mongoDB[collection_1].find(query)))
    df_2 = pd.DataFrame(list(mongo.mongoDB[collection_2].find(query)))
    df = pd.merge(df_1, df_2, on="CurTime")
    df.loc[df["Side"] == "Bid", "Dif"] = (df["Price"] - df["Last"])
    df.loc[df["Side"] == "Ask", "Dif"] = (df["Last"] - df["Price"])
    df.loc[df["Side"] == "Ask", "TotalSize"] *= -1
    df = df.loc[df["Dif"] >= price_dif_threshold]
    sum_df = df.groupby("CurTime")["TotalSize"].sum()
    final_df = pd.merge(sum_df, df_1, on="CurTime")[["TradeTime", "Last", "TotalSize"]]
    final_df = final_df.rename(columns={"TotalSize": "VolumeSum"})
    final_df["LastDif"] = final_df["Last"].diff().shift(-1)
    final_df["SameSign"] = (final_df["VolumeSum"] > 0) == (final_df["LastDif"] > 0)
    # print(final_df["SameSign"].sum())
    # print(len(final_df))
    final_df.loc[final_df["VolumeSum"]>=size_threshold, "Gain"] = final_df["LastDif"]
    final_df.loc[final_df["VolumeSum"]<size_threshold, "Gain"] = 0
    final_df["Gain"] = (final_df["Gain"] * 100).apply(np.floor) / 100
    print(final_df)
    final_df.to_csv(f"tradestation_data/tsla_{date}_gain.csv")
    return final_df["Gain"].sum()
symbol = "TSLA"
date = "2023-06-07"
#for price_dif_threshold in np.arange(-.005, -0.03, -.001):
price_dif_threshold = -.015
size_threshold = 0
print(f"{price_dif_threshold} {size_threshold} {calc_gain(symbol, date, price_dif_threshold, size_threshold)}")


NameError: name 'mongo' is not defined

In [ ]:
def read_lvl1(symbol, time):
    collection = f"{symbol}_10sec_ts_lvl1"
    query = {"CurTime": time}
    lvl1 = None
    while lvl1 is None:
        lvl1 = mongo.mongoDB[collection].find_one(query)
        print(lvl1)
    return None
symbol = "TSLA"
time = "2023-06-07T18:56:40Z"
print(read_lvl1(symbol, time))

In [153]:
import datetime
timenow = datetime.datetime.utcnow()
print(timenow)
timenow = timenow.replace(second=timenow.second - timenow.second % 10)
print(timenow)

2023-06-07 19:10:33.328367
2023-06-07 19:10:30.328367


In [5]:
import requests
account_id = "11655345"
def buy_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "BUY",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
buy_share("TSLA")

{"Orders":[{"Message":"Sent order: Buy 1 TSLA @ Market","OrderID":"967793406"}]}



In [11]:
import requests
def sell_share(symbol):
    access_token = refresh()

    url = "https://api.tradestation.com/v3/orderexecution/orders"

    payload = {
        "AccountID": "11655345",
        "Symbol": f"{symbol}",
        "Quantity": "1",
        "OrderType": "Market",
        "TradeAction": "SELL",
        "TimeInForce": {"Duration": "DAY"},
        "Route": "Intelligent"
    }
    headers = {
        "content-type": "application/json",
        "Authorization": f"Bearer {access_token}"
    }

    response = requests.request("POST", url, json=payload, headers=headers)

    print(response.text)
sell_share("TSLA")

{"Orders":[{"Message":"Sent order: Sell 1 TSLA @ Market","OrderID":"967802392"}]}



In [13]:
import requests
def check_status():
    access_token = refresh()
    url = "https://api.tradestation.com/v3/brokerage/accounts/11655345/orders"

    headers = {"Authorization": f"Bearer {access_token}"}

    response = requests.request("GET", url, headers=headers)

    print(response.text)
check_status()

{"Orders":[{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:10Z","Currency":"USD","Duration":"DAY","FilledPrice":"0","Legs":[{"QuantityOrdered":"1","ExecQuantity":"0","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK"}],"OrderID":"967805697","OpenedDateTime":"2023-06-07T19:53:10Z","OrderType":"Market","PriceUsedForBuyingPower":"224.98","RejectReason":"EC501: Day trading margin rules, this trade can possibly designate you as a pattern day trader, please call the Trade Desk at 800-871-3563 for more information.","Routing":"Intelligent","Status":"REJ","StatusDescription":"Rejected","UnbundledRouteFee":"0"},{"AccountID":"11655345","CommissionFee":"0","ClosedDateTime":"2023-06-07T19:53:00Z","Currency":"USD","Duration":"DAY","FilledPrice":"224.9699","Legs":[{"OpenOrClose":"Open","QuantityOrdered":"1","ExecQuantity":"1","QuantityRemaining":"0","BuyOrSell":"Buy","Symbol":"TSLA","AssetType":"STOCK","ExecutionPrice":"224.9699"}],"OrderID":